In [5]:
pip install tensorflow
pip install keras
pip install sys
pip install os
pip install

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf
import numpy as np
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt
import sys
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, BatchNormalization, Dropout, Flatten, Dense, Activation, Conv2D, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import time

start = time.time()

In [2]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS 'C:/Users/Rishabh Patni/Desktop/Projects/Keras Coin-eaf1b6df7b08.json'

env: GOOGLE_APPLICATION_CREDENTIALS='C:/Users/Rishabh Patni/Desktop/Projects/Keras Coin-eaf1b6df7b08.json'


In [3]:
#Variables
IMG_WIDTH, IMG_HEIGHT = 400, 400
BATCH_SIZE = 173
samples_per_epoch = 6228
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
nb_filters3 = 128
nb_filters4 = 256
conv1_size = 7
conv2_size = 5
conv3_size = 3
conv4_size = 3
conv5_size = 1
pool_size = 2
classes_num = 211
lr = 0.01
epochs = 200
decay_rate = lr/epochs

In [4]:
#Validation Set
image_generator = ImageDataGenerator(rescale=1./255)


data_dir_2 = "C:/Users/Rishabh Patni/Desktop/Projects/Coin-Identifier/Training Images/validation/"
data_2 = pathlib.Path(data_dir_2)
CLASS_NAMES_1 = np.array([item.name for item in data_2.glob('dir/*')])
validation_data_gen = image_generator.flow_from_directory(directory = data_dir_2, 
                                                     batch_size = BATCH_SIZE, 
                                                     shuffle = False, 
                                                     classes = list(CLASS_NAMES_1),
                                                     target_size = 
                                                     (IMG_HEIGHT, IMG_WIDTH), 
                                                     color_mode = "rgb")

NameError: name 'ImageDataGenerator' is not defined

In [5]:
#Training Set
image_generator_training = ImageDataGenerator(rescale=1./255, 
                                     rotation_range=10,
                                     zoom_range=0.05)

data_dir = "C:/Users/Rishabh Patni/Desktop/Projects/Coin-Identifier/Training Images/train/"
data = pathlib.Path(data_dir)
CLASS_NAMES = np.array([item.name for item in data.glob('dir/*')])
train_data_gen = image_generator_training.flow_from_directory(directory = data_dir, 
                                                     batch_size = BATCH_SIZE, 
                                                     shuffle = True, 
                                                     classes = list(CLASS_NAMES),
                                                     target_size = 
                                                     (IMG_HEIGHT, IMG_WIDTH), 
                                                     color_mode = "rgb")

Found 6228 images belonging to 211 classes.


In [6]:
## Model Development ##
# First Layer (Visible)
visible = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))

# Second Layer
a= Conv2D(32, kernel_size = (3, 3), 
          padding ="same") (visible)
a= Activation("relu")(a)
a = MaxPooling2D(pool_size=(2, 2)) (a)

shortcut_a2 = a

# Second Layer (re-run with 32 filters)
a = Conv2D(32, kernel_size = (2, 2), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)

# Second Layer (re-run with 32 filters)
a = Conv2D(32, kernel_size = (2, 2), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)
a = keras.layers.concatenate([a, shortcut_a2])
a = MaxPooling2D(pool_size=(2, 2)) (a)



# Third Layer (64 Filters)
a = Conv2D(64, kernel_size = (3, 3), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)

shortcut_a3 = a

# Third Layer (Re-run with 64 Filters)
a = Conv2D(64, kernel_size = (3, 3), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)

# Third Layer (Re-run with 64 Filters)
a = Conv2D(64, kernel_size = (3, 3), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)
a = keras.layers.concatenate([a, shortcut_a3])
a = MaxPooling2D(pool_size=(2, 2)) (a)



# Fourth Layer (128 Filters)
a = Conv2D(128, kernel_size = (3, 3), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)

shortcut_a4 = a

# Fourth Layer (Re-run with 128 Filters)
a = Conv2D(128, kernel_size = (3, 3), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu") (a)

# Fourth layer (Re-run with 128 Filters)
a = Conv2D(128, kernel_size = (2, 2), padding ="same") (a)
a = BatchNormalization()(a)
a = Activation("relu")(a)
a = keras.layers.concatenate([a, shortcut_a4])
a = MaxPooling2D(pool_size=(2, 2), strides = None) (a)

# Consolidating and Output
a = Flatten()(a)
output = Dense(classes_num, activation='softmax') (a)
model = Model(inputs = visible, outputs = output)
print(model.summary())

#Defining Optimizer
Optimize = keras.optimizers.Adam (learning_rate = lr, decay = decay_rate)
model.compile(loss = 'categorical_crossentropy',
              optimizer = Optimize, 
              metrics = ['accuracy'])

# Creating Checkpoint measure
filepath = 'C:/Users/Rishabh Patni/Desktop/Projects/Coin-Identifier/V7/coin.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_weights_only = False,
                                             verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 400, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 400, 400, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 200, 200, 32) 0           activation_1[0][0]               
____________________________________________________________________________________________

In [1]:
#fitting data to model
model.fit_generator(
    train_data_gen,
    epochs=epochs,
    validation_data=validation_data_gen,
    callbacks = callbacks_list, 
    verbose = 1
)

#Save model
model.save('C:/Users/Rishabh Patni/Desktop/Projects/Coin-Identifier/V7/coin.h5')
model.save_weights('C:/Users/Rishabh Patni/Desktop/Projects/Coin-Identifier/V7/coinweights.h5')

#Calculate execution time
end = time.time()
dur = end-start
print(dur)

NameError: name 'model' is not defined

In [35]:
# testing set
data_dir_3 = "C:/Users/Rishabh Patni/Desktop/Projects/Coin-Identifier/Training Images/test/"
data_3 = pathlib.Path(data_dir_3)
CLASS_NAMES_2 = np.array([item.name for item in data_3.glob('dir/*')])
test_data_gen = image_generator.flow_from_directory(directory = data_dir_3, 
                                                     batch_size = BATCH_SIZE, 
                                                     shuffle = True, 
                                                     classes = list(CLASS_NAMES_2),
                                                     target_size = 
                                                     (IMG_HEIGHT, IMG_WIDTH), 
                                                     color_mode = "rgb")

Found 844 images belonging to 211 classes.


In [38]:
#testing
results = model.evaluate(test_data_gen)
print('test loss, test acc:', results)

14/14 [==============================] - 18s 1s/step
test loss, test acc: [1.9074764251708984, 0.7523696422576904]


In [7]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

env: GOOGLE_APPLICATION_CREDENTIALS=''
